In [ ]:
import os
import win32com.client
import numpy as np
import re

def replace_D(s, new_number):
    # Reverse the string to process from the back
    reversed_s = s[::-1]
    # Find the positions of the operators in the reversed string
    operator_positions = [m.start() for m in re.finditer(r'[\+\-]', reversed_s)]
    # The position of the second operator from the end in the original string
    second_operator_pos = operator_positions[1]
    # Reverse the position to get the correct index in the original string
    index_in_original = len(s) - second_operator_pos - 1
    # Find the end position of the number following the second operator
    next_operator_match = re.search(r'[\+\-]', s[index_in_original+1:])
    if next_operator_match:
        end_pos = index_in_original + next_operator_match.start() + 1
    else:
        end_pos = len(s)
    # Replace the second-to-last element including its sign
    new_string = s[:index_in_original] + str(new_number) + s[end_pos:]
    return new_string

#define groups through which to access all utility functions
VHG_EinP = ['EinpA_mF','EinpA_oF','EinpF_mF','EinpF_oF','EinpK_mF','EinpK_oF','EinpU_mF','EinpU_oF']
Mode = ['bic']

VHG_main = [
"BSch_mF", "BSch_oF", "ERWmP_mF", "ERWmP_oF", "ERWoP_mF", "ERWoP_oF", "GSch_mF", "GSch_oF", "Kind_mF", "Kind_oF",
"NERWmP_mF", "NERWmP_oF", "NERWoP_mF", "NERWoP_oF", "Rentner_mF", "Rentner_oF", "Sch_mF", "Sch_ko", "Stud_mF", "Stud_oF"
]
Activity = ["A", "Ax", "B", "E", "F", "Fx", "G", "H", "J", "K", "Kx","S", "U", "Ux", "W", "Y"]

#Parameter values for D to try out (see Excel 003)
D = [['+2.5','+2.6','+2.55'],
     ['+2.7','+2.9','+2.75'],
     ['+1.9','+1.9','+1.9'],
     ['+0.6','+0.6','+0.6'],
     ['+1.1','+1.2','+1.2'],
     ['+2','+2.2','+2.1'],
     ['+2.4','+2.5','+2.45'],
     ['+1.8','+1.8','+1.8'],
     ['+3.4','+3.6','+3.5'],
     ['+4','+4.2','+4.1']]

# Matrix numbers for calculating resulting modal share per person group
mBSch_bic = 5243
mBSch_other = [5241, 5242, 5244, 5245]

mERWmP_bic = 5248
mERWmP_other = [5246, 5247, 5249, 5250]

mERWoP_bic = 5253
mERWoP_other = [5251, 5252, 5254, 5255]

mNERWmP_bic = 5268
mNERWmP_other = [5266, 5267, 5269, 5270]

mNERWoP_bic = 5273
mNERWoP_other = [5271, 5272, 5274, 5275]

mGSchSch_bic = [5258, 5283]
mGSchSch_other = [5256, 5257, 5259, 5260, 5281, 5282, 5284, 5285]

mStud_bic = 5288
mStud_other = [5286, 5287, 5289, 5290]

mRentner_bic = 5278
mRentner_other = [5276, 5277, 5279, 5280]

mKind_bic = 5263
mKind_other = [5261, 5262, 5264, 5265]

mEinp_bic = 42
mEinp_other = [39, 40, 41]

In [ ]:
#Loop over iteration steps to change D, rerun model, compute mode shares by distance classes, and save
for i in range(2,3):
    verPath = r'C:\Users\arning.FUL\Desktop\ver\DEV_20250116_0845_D3_0.ver'
    #verPath = r'D:\ver\20240709_1200.ver'
    Visum = win32com.client.Dispatch('Visum.Visum.24')
    Visum.loadVersion(verPath)
    
    #change utils for Einpendler visem
    visem_einp = Visum.Procedures.Operations.ItemByKey(44)
    for mode in Mode:
        for vhg in VHG_EinP:
            
            #get old utilityfunction
            util = visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).AttValue("UtilityFunction")
            
            #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
            if util != "-99999":
                if mode == "bic":
                    new_D = D[9][i]

                # Set D for all Einp utility functions to value from D (not calibrated, only set similar to value of other groups
                new_util = replace_D(util,new_D)
                #write into Visum
                visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).SetAttValue("UtilityFunction", new_util)
                
    #change utils for main visem
    visem_main = Visum.Procedures.Operations.ItemByKey(45)
    for mode in Mode:
        for vhg in VHG_main:
            for activity in Activity:
                #get old utility function
                #not all combinations of vhg and activities exist; skip those exception
                try:
                    util = visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).AttValue("UtilityFunction")
                    if util != "-99999":
                        if mode == "bic":
                            if vhg =="ERWmP_mF" or vhg =="ERWmP_oF":
                                new_D = D[0][i]
                            elif vhg =="ERWoP_mF" or vhg =="ERWoP_oF":
                                new_D = D[1][i]
                            elif vhg =="NERWmP_mF" or vhg =="NERWmP_oF":
                                new_D = D[2][i]
                            elif vhg =="NERWoP_mF" or vhg =="NERWoP_oF":
                                new_D = D[3][i]
                            elif vhg =="BSch_mF" or vhg =="BSch_oF":
                                new_D = D[4][i]
                            elif vhg =="GSch_mF" or vhg =="Sch_mF" or vhg =="GSch_oF" or vhg =="Sch_oF":
                                new_D = D[5][i]
                            elif vhg =="Stud_mF" or vhg =="Stud_oF":
                                new_D = D[6][i]
                            elif vhg =="Rentner_mF" or vhg =="Rentner_oF":
                                new_D = D[7][i]
                            elif vhg =="Kind_mF" or vhg =="Kind_oF":
                                new_D = D[8][i]
                                
                        #write new utilityfunction
                        new_util = replace_D(util,new_D)
                        #write into Visum
                        visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).SetAttValue("UtilityFunction", new_util)
                except Exception as e:
                    pass
                    # handle exception (skip the current iteration)
                    # print(f"Skipping vhg: {vhg}, activity: {activity}, mode: {mode} due to error: {e}")
    #run procedures
    Visum.Procedures.Execute() 
    
    # Calculate mode share per persongroup
    print('THIS IS FOR ITERATION NUMBER '+str(i))
    print('mERWmP')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mERWmP_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mERWmP_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWmP_other[3]).GetValues()).sum())*100, 1))
    print('')
    
    print('mERWoP')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mERWoP_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mERWoP_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mERWoP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mNERWmP')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWmP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mNERWoP')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mNERWoP_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mBSch')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mBSch_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mBSch_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mBSch_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mGSchSch')
    print(round((np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_bic[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_bic[1]).GetValues()).sum())/
          (np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_bic[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_bic[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[3]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[4]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[5]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[6]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mGSchSch_other[7]).GetValues()).sum())*100, 1))
    print('')

    print('mStud')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mStud_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mStud_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mStud_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mRentner')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mRentner_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mRentner_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mRentner_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mKind')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mKind_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mKind_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[2]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mKind_other[3]).GetValues()).sum())*100, 1))
    print('')

    print('mEinp')
    print(round(np.array(Visum.Net.Matrices.ItemByKey(mEinp_bic).GetValues()).sum()/
          (np.array(Visum.Net.Matrices.ItemByKey(mEinp_bic).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[0]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[1]).GetValues()).sum()+
           np.array(Visum.Net.Matrices.ItemByKey(mEinp_other[2]).GetValues()).sum())*100, 1))
    print('')
    
    #save
    #verPath_append = r'D:\ver\20240709_1200.ver'#+str(i)+'.ver'
    verPath_append = r'C:\Users\arning.FUL\Desktop\ver\DEV_20250116_0845_D3_'+str(i)+'.ver'
    #Visum.saveVersion(verPath_append)
    Visum = None